<a href="https://colab.research.google.com/github/iliaadam/Eurostat-KG-Enrichment-Thesis-2023/blob/main/ThesisEurostatEnrichment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install SPARQLWrapper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.4 MB/s eta 0:00:00


In [ ]:
from SPARQLWrapper import SPARQLWrapper, TURTLE
import re

Συνάρτηση που παράγει δυο dictionaries με values τα references:

a) ένα με key τους τίτλους, title_to_refs

b) ένα με key τα urls, url_to_refs  

In [ ]:
def get2Diction(q,q2):
  # Set up the SPARQL endpoint URL
  endpoint_url = "http://lod.csd.auth.gr:8890/sparql/"
  # Create a new SPARQLWrapper object and set its endpoint URL
  sparql = SPARQLWrapper(endpoint_url)
  # Set the query string in the SPARQLWrapper object
  sparql.setQuery(q)
  # Set the return format to Turtle
  sparql.setReturnFormat(TURTLE)
  # Execute the query and get the results as a string in Turtle format
  result = sparql.query().response.read().decode()
  # Extract the local names from the subject URIs
  db_refs = re.findall(r"ns2:(\w+)", result)
  # Extract the title
  db_titles = re.findall(r'"(.+?)"\^\^xsd:string', result)
  # Create a new SPARQLWrapper object and set its endpoint URL
  sparql = SPARQLWrapper(endpoint_url)
  # Set the query string in the SPARQLWrapper object
  sparql.setQuery(q2)
  # Set the return format to Turtle
  sparql.setReturnFormat(TURTLE)
  # Execute the query and get the results as a string in Turtle format
  result = sparql.query().response.read().decode()
  db_urls = re.findall(r'"(.+?)"\^\^xsd:anyURI', result)

  title_to_refs = {} #dictionary for the distinct titles
  for i, title in enumerate(db_titles):
      if title not in title_to_refs:
          title_to_refs[title] = set([db_refs[i]])
      else:
          title_to_refs[title].add(db_refs[i])

  url_to_refs = {} #dictionary for the distinct urls of each title
  for i, url in enumerate(db_urls):
      if url not in url_to_refs:
          url_to_refs[url] = set([db_refs[i]])
      else:
          url_to_refs[url].add(db_refs[i])

  return title_to_refs , url_to_refs


Sparql ερωτήματα για την ανάκτηση των references, τίτλων, urls τόσο απο την κλάση Table όσο και απο την κλάση Database.

In [ ]:
query="""
PREFIX estat: <https://ec.europa.eu/eurostat/NLP4StatRef/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
construct{
    ?r estat:title ?t .
} where {
    ?r rdf:type estat:Table .
    ?r estat:title ?t .
}
"""

query2 = """
PREFIX estat: <https://ec.europa.eu/eurostat/NLP4StatRef/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
construct{
    ?r estat:hasURL ?c .
} where {
    ?r rdf:type estat:Table .
     ?r estat:hasURL ?c .
}
"""

tableTTR,tableUTR = get2Diction(query,query2)

query="""
PREFIX estat: <https://ec.europa.eu/eurostat/NLP4StatRef/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
construct{
    ?r estat:title ?t .
} where {
    ?r rdf:type estat:Database .
    ?r estat:title ?t .
}
"""

query2 = """
PREFIX estat: <https://ec.europa.eu/eurostat/NLP4StatRef/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
construct{
    ?r estat:hasURL ?c .
} where {
    ?r rdf:type estat:Database .
     ?r estat:hasURL ?c .
}
"""

dbTTR,dbUTR = get2Diction(query,query2)

Συνάρτηση η οποία επιστρέφει τα labels (l_values) και τους κωδικούς (c_values) (επεξεργασμένα καταλλήλως), των κλάσεων tb και data.

In [ ]:
def getLCvals(q):
  # Set up the SPARQL endpoint URL
  endpoint_url = "http://lod.csd.auth.gr:8890/sparql/"

  # Create a new SPARQLWrapper object and set its endpoint URL
  sparql = SPARQLWrapper(endpoint_url)
  # Set the SPARQL query
  query = q
  # Set the query string in the SPARQLWrapper object
  sparql.setQuery(query)

  # Set the return format to Turtle
  sparql.setReturnFormat(TURTLE)

  # Execute the query and get the results as a string in Turtle format
  result = sparql.query().response.read().decode()

  # Extract the local names from the subject URIs
  c_values = re.findall(r"ns2:(\w+[\w-]*)", result)

  # Extract the local names from the subject URIs
  l_values = re.findall(r'"(.+?)"\^\^xsd:string', result)
  return l_values , c_values

Sparql ερωτήματα για την ανάκτηση όλων των labels και κωδικών απο τις υποκλάσεις των κλάσεψν data και tb.

In [ ]:
query = '''
PREFIX estat: <https://ec.europa.eu/eurostat/NLP4StatRef/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

CONSTRUCT{
   ?c rdfs:label ?l .
}WHERE {
  ?c rdfs:subClassOf* estat:tb .
  ?c rdfs:label ?l .
  FILTER(?c != estat:tb)
}
'''
tlv , tcv = getLCvals(query)

query = '''
PREFIX estat: <https://ec.europa.eu/eurostat/NLP4StatRef/ontology/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

CONSTRUCT{
   ?c rdfs:label ?l .
}WHERE {
  ?c rdfs:subClassOf* estat:data .
  ?c rdfs:label ?l .
  FILTER(?c != estat:data)
}
'''
dlv , dcv = getLCvals(query)

Συνάρτηση η οποία δέχεται ένα url και επιστρέφει άν βρεί κάποιον κωδικό που πιθανώς να υπάρχει μέσα σε αυτό

In [ ]:
def getCode(s):
    match = re.search(r'code=(.*?)&', s)
    if match:
        return match.group(1)
    match =  re.search(r'dataset=(.*?)&', s)
    if match:
        return match.group(1)
    match =  re.search(r'view/(.*?)/default/table?lang=en', s)
    if match:
        return match.group(1)
    match = re.search(r"(?<=view\/).+?(?=\/default\/table\?lang=en)",s)
    if match:
        return match.group(0)
    if "?node_code=" in s:
        node_code = s.split("?node_code=")[1]
        return node_code
    if "?mode=view&code=" in s:
      code = s.split("?mode=view&code=")[1]
      return code
    match =  re.search(r'bookmark/(.*?)?lang=en', s)
    if match:
        return match.group(1)
    if "/-/" in s:
      code = s.split("/-/")[1]
      return code
    match =  re.search(r'web/(.*?)/data/', s)
    if match:
      code=match.group(1)
      if "/" in code:
        parts = code.split("/")
        code = parts[-1]
      return code
    match =  re.search(r'web/(.*?)/database', s)
    if match :
      code=match.group(1)
      if "/" in code:
         parts = code.split("/")
         code = parts[-1]
      return code
    match = re.search(r'web/(.*?)/main-tables', s) #for tables
    if match :
      code=match.group(1)
      if "/" in code:
         parts = code.split("/")
         code = parts[-1]
      return code

    match =  re.search(r'web/(.*?)/overview', s)
    if match:
      code=match.group(1)
      return code
    if "/web/" in s :
      code = s.split("/web/")[1]
      if "/" in code:
        parts = code.split("/")
        code = parts[-1]
      return code

    return " "

In [ ]:
def get_key_by_value(dict_obj, value):
    return [key for key in dict_obj if value in dict_obj.get(key)]

def createTTC(title_to_refs,url_to_refs): #creates dictionary with keys the titles and values the codes
  title_to_codes = dict()
  for key in title_to_refs.keys():
      title_to_codes[key] = []
      for value in title_to_refs[key]:
          new_val =get_key_by_value(url_to_refs,value)
          code = getCode(new_val[0])
          if code not in title_to_codes.get(key):
            title_to_codes[key].append(code)
  return title_to_codes

αρχικά χρεισιμοποιώ data and database

In [ ]:
title_to_refs , url_to_refs = dbTTR , dbUTR
title_to_codes = createTTC(title_to_refs,url_to_refs)
l_values , c_values = dlv , dcv

Η φάση Α, εξετάζει πλήρη ταυτοποίηση label-τίτλου ή κωδικών, είτε πριν την επεξεργασία των δεδομένων είτε μετά (χωρίς την χρήση αλγορίθμων υπολογισμού ομοιότητας).

In [ ]:
def phaseA(titles,codeDict,labels):
  changes=[]
  remaining=list(titles)
  with open("phaseA.txt", "w") as f:
    for i,title in enumerate(titles):
      t = title.lower()
      split_string = t.split("  ") # Split the string based on double space
      if len(split_string) >= 2:
          new_title = split_string[0] # Extract the first element of the list
          num = len(split_string[1].split())
      else:
          new_title = t
          num = 0
      split_string = t.split("   ")  # Split the string based on triple space
      cut = split_string[0]  # Extract the first element of the list
      new_title2 = re.sub(r' +', ' ', cut)
      new_title3 = re.sub(r'\s+', ' ', t)   #original title but without multiple spaces in a row
      match = re.search(r"\s{2,}(.+)", t) # test if the title have 2 or more white spaces
      if match:
        new_title4 = match.group(1) # get whatever is after the multiple spaces
        new_title5 = re.sub(r"\s{2,}", " ", new_title4) # convert every multiple spaces in a row to a single space
      else:
        new_title4 = t
        new_title5 = t
      code1 = codeDict.get(title)
      code = code1[0]
      for j,label in enumerate(labels):
        l=label.lower()
        label_code = c_values[j]
        if (t == l) or (label_code == code) or (new_title == l and num < 5) or (new_title2 == l) or (new_title3 == l) or (l==new_title4) or (l==new_title5):
          f.write("{} : {} \n".format(title, label))
          #ref=list(title_to_refs.get(title))
          #ref_url ="http://lod.csd.auth.gr:7200/resource?uri=https://ec.europa.eu/eurostat/NLP4StatRef/knowledge/"+ref[0]
          #data_url ="http://lod.csd.auth.gr:7200/resource?uri=https://ec.europa.eu/eurostat/NLP4StatRef/knowledge/"+label_code
          #f.write("{}({}) : {}({})\n".format(title, ref_url, label, data_url))
          changes.append([title,c_values[j]])
          remaining.remove(title)
  return changes , remaining


In [ ]:
changes , remaining = phaseA(title_to_refs.keys(),title_to_codes,l_values)

Συναρτήσεις φιλτραρίσματος συμβολοσειρών (πχ stem, remove stopwords, lowercase κτλ.)

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

def filtering3(text):
  return text.lower()

def filtering2(text): #main
  # Initialize the stemmer and download the stopwords
  stemmer = PorterStemmer()
  stop_words = set(stopwords.words('english'))
  # Tokenize the text into words
  words = word_tokenize(text)
  # Remove the stop words from the words list
  filtered_words = [word for word in words if word.lower() not in stop_words]
  # Perform stemming on the remaining words
  stemmed_words = [stemmer.stem(word) for word in filtered_words]
  # Join the stemmed words into a string
  stemmed_text = " ".join(stemmed_words)
  return stemmed_text

def filtering(text):
  stop_words = set(stopwords.words('english'))
  # Tokenize the text into words
  words = word_tokenize(text)
  # Remove the stop words from the words list
  filtered_words = [word for word in words if word.lower() not in stop_words]
  return " ".join(filtered_words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Στην συνέχεια υπάρχουν διάφοροι αλγόριθμοι και τεχνικές υπολογισμού ομοιότητας μεταξύ συμβολοσειρών, όμως τελικά χρησιμοποιούνται μόνο οι: **jellyfish, fuzzywuzzy, word2vec και tfidf**.

**SPACY**

In [ ]:
import spacy
!python -m spacy download en_core_web_md
#!python -m spacy download en_core_web_lg

2023-04-05 20:35:45.032273: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 18.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
#nlp = spacy.load("en_core_web_lg")
nlp = spacy.load("en_core_web_md")

def spacySim(title,label):
  doc=nlp(title)
  doc2=nlp(label)
  return doc.similarity(doc2)

In [ ]:
docc=nlp("Employment and unemployment  LFS")
docc2=nlp("Employment and unemployment Labour force survey")
print(docc.similarity(docc2))

0.8341739918712523


**fuzzywuzzy**

In [ ]:
!pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import fuzz

def fuzzySim(title,label):
  return fuzz.ratio(title, label)/100.0

**jellyfish**

In [ ]:
!pip install jellyfish

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.9 MB/s eta 0:00:00


In [ ]:
import jellyfish

def jellyfishSim(title,label):
  return jellyfish.jaro_winkler(title, label)

**word2vec**

In [ ]:
from gensim.models import Word2Vec
import numpy as np
def trainW2V(l_values):
  sentences = []
  for row in l_values: # creating the vocabulary
    sentences.append(filtering2(row))
  vocabulary = [row.split() for row in sentences]
  # Train the Word2Vec model
  model = Word2Vec(vocabulary, vector_size=2000, window=3, min_count=1, sample=1e-4, epochs=20, sg=0, batch_words=1000, alpha=0.03,seed = 23)
  return model

In [ ]:
# Compute the vector representation for a sentence by taking the average of the word vectors
def sentence_vector(sentence):
    words = sentence.split()
    vector= [model.wv.get_vector(token) for token in words if token in model.wv.key_to_index]
    if len(vector) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vector, axis=0)

# Compute the similarity between two sentences
def sentence_similarity(sentence1, sentence2):
    vec1 = sentence_vector(sentence1)
    vec2 = sentence_vector(sentence2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def myWord2vec(title,label):
  return sentence_similarity(title,label)

In [ ]:
from nltk.tokenize import word_tokenize

def jaccard_similarity(sentence1, sentence2):
    # Tokenize the sentences into sets of words
    set1 = set(word_tokenize(sentence1.lower()))
    set2 = set(word_tokenize(sentence2.lower()))

    # Compute the Jaccard similarity
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)



**pretrained word2vec**

In [ ]:
import gensim.downloader as api
from gensim.models import KeyedVectors
from scipy import spatial
import numpy as np

# Load the pre-trained Word2Vec model
model_name = 'word2vec-google-news-300'
wv = api.load(model_name)

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
# Compute the sentence similarity using cosine similarity of word embeddings
def pretrainedWord2vec(s1, s2):
    s1_vec = sum([wv[w] for w in s1.split() if w in wv])
    s2_vec = sum([wv[w] for w in s2.split() if w in wv])

    # Check if vectors are non-zero
    if np.count_nonzero(s1_vec) == 0 or np.count_nonzero(s2_vec) == 0:
        return 0.0

    similarity = 1 - spatial.distance.cosine(s1_vec, s2_vec)
    return similarity


**fasttext** (perform better on tasks that involve rare words or misspelled words )

In [ ]:
import gensim.downloader as api
from gensim.models import KeyedVectors
from scipy import spatial
import numpy as np

# Load the pre-trained FastText model
model_name = 'fasttext-wiki-news-subwords-300'
wv = api.load(model_name)                                           # Compute the sentence similarity using cosine similarity of word embeddings
def fastText(s1, s2):
    s1_vec = sum([wv[w] for w in s1.split() if w in wv])
    s2_vec = sum([wv[w] for w in s2.split() if w in wv])

    # Check if vectors are non-zero
    if np.count_nonzero(s1_vec) == 0 or np.count_nonzero(s2_vec) == 0:
        return 0.0

    similarity = 1 - spatial.distance.cosine(s1_vec, s2_vec)
    return similarity


[=================================================-] 99.9% 957.5/958.4MB downloaded


**Transformers**

In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer, util


model = SentenceTransformer('distilbert-base-nli-mean-tokens')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.0 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=dc11453ad13b68ba8aa6bf022d53eb933e71a857c24137b55c689cacd64bc3a2
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [ ]:
def transformerSim(sentence1,sentence2):
  query_embedding = model.encode(sentence1)
  passage_embedding = model.encode(sentence2)
  #return util.dot_score(query_embedding, passage_embedding).item()
  return util.cos_sim(query_embedding, passage_embedding).item()

In [ ]:
print(transformerSim("Employment and unemployment  LFS" ,"Employment LFS series"))
print(transformerSim("Employment and unemployment  LFS" , "Employment and unemployment Labour force survey"))

0.8674449920654297
0.8890995383262634


**TF-IDF**

In [ ]:
import numpy as np
def createVec(titles,l_values):
  sentences = []
  for row in titles:
    sentences.append(filtering2(row))
  for row in l_values:
    sentences.append(filtering2(row))
  words = set([word for sentence in sentences for word in sentence.split()])
  return sentences,list(words)

sentences , distinct_words = createVec(title_to_refs.keys(),l_values)

def vectorize(sentence):
  f=list()
  words = sentence.split()
  for i,word in enumerate(sentence):
    f.append(sentence.count(word))
  maxF=max(f)
  vector=list()
  for i,word in enumerate(distinct_words):
    if word not in words:
      vector.append(0)
    else:
      n = sum(1 for s in sentences if word in s)
      tf=sentence.count(word)/maxF
      idf=np.log(len(sentences)/n)
      vector.append(tf*idf)
  return vector

def tfIdfSim(sentence1,sentence2):
  vec1=vectorize(sentence1)
  vec2=vectorize(sentence2)
  cosine = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
  return cosine

In [ ]:
print(tfIdfSim(filtering("Health hlt") , filtering("Health")))

0.875


**Doc2vec**

In [ ]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from scipy.spatial.distance import cosine
sentences = []
for row in title_to_refs.keys():
  sentences.append(filtering(row))
for row in l_values:
  sentences.append(filtering(row))
documents = []
for i, sentence in enumerate(sentences):
    words = gensim.utils.simple_preprocess(sentence)
    documents.append(TaggedDocument(words, [i]))


model = Doc2Vec(vector_size=50, window=5, min_count=1, epochs=10, workers=4)
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

def doc2vecSim(sentence1,sentence2):
  tokens1 = gensim.utils.simple_preprocess(sentence1)
  tokens2 = gensim.utils.simple_preprocess(sentence2)

  vector1 = model.infer_vector(tokens1)
  vector2 = model.infer_vector(tokens2)

  return 1 - cosine(vector1, vector2)


Φάση Β, υπολογισμός ομοιοτήτων με την χρήση των προηγούμενων αλγορίθμων

In [ ]:
from tqdm import tqdm
def highestSimilarity(title,func,useCode,filtering):
  max=0
  index=0 # it doesent matter , this variable will be used only if max is greater than threshold which means that variable max will have a value
  titleF = filtering(title)
  code = title_to_codes.get(title)
  code = re.sub(r'\d+', '', code[0]) #remove numeric characters from code
  for i,l in enumerate(l_values):
    if useCode == 0 or code== ' ':
      sim = func(titleF,filtering(l))
    elif useCode == 1:
      sim = jellyfishSim(re.sub(r'\d+', '',c_values[i]),code)
    else:
      sim = (1-useCode)*func(titleF,filtering(l)) + useCode*jellyfishSim(re.sub(r'\d+', '',c_values[i]),code)
    if sim >= 1 : # sometimes it get little over 1 due to floating-point rounding errors, numerical instability, or other factors
      return 1 , i
    elif sim > max:
      max=sim
      index=i
  return max , index #return the maximum similarity and the index that have this similarity

def calculateSim(func,threshold,titles,useCode,filtering,file):
  changes=[]
  remaining=list(titles)
  with open(file, "a") as f:
    for i,t in enumerate(tqdm(titles)):
      sim , index = highestSimilarity(t,func,useCode,filtering) #filtering(t)
      if sim > threshold:
        changes.append([t,c_values[index]])
        remaining.remove(t)
        f.write("{} : {} ------> {:.3f}\n".format(t, l_values[index], sim))
        #ref=list(title_to_refs.get(t))
        #ref_url ="http://lod.csd.auth.gr:7200/resource?uri=https://ec.europa.eu/eurostat/NLP4StatRef/knowledge/"+ref[0]
        #data_url ="http://lod.csd.auth.gr:7200/resource?uri="+c_values[index]
        #f.write("{}({}) : {}({}) ------> {:.3f}\n".format(t, ref_url, l_values[index], data_url, sim))
  return changes , remaining



Database-data

In [ ]:
changes , remaining = phaseA(title_to_refs.keys(),title_to_codes,l_values)

In [ ]:
sentences,distinct_words= createVec(title_to_refs.keys(),l_values)
changes2 , remaining2 = calculateSim(tfIdfSim,0.8,remaining,0,filtering2,"phaseB.txt")

In [ ]:
changes3 , remaining3 = calculateSim(jellyfishSim,0.9,remaining2,0,filtering3,"phaseB.txt")

In [ ]:
model = trainW2V(l_values)
changes4 , remaining4 = calculateSim(myWord2vec,0.8,remaining3,0,filtering2,"phaseB.txt")

In [ ]:
changes5 , remaining5 = calculateSim(fuzzySim,0.74,remaining4,0.25,filtering3,"phaseB.txt")

Table-tb

In [ ]:
title_to_refs , url_to_refs = tableTTR , tableUTR
title_to_codes = createTTC(title_to_refs,url_to_refs)
l_values , c_values = tlv , tcv

In [ ]:
changes , remaining = phaseA(title_to_refs.keys(),title_to_codes,l_values)

In [ ]:
changes2 , remaining2 = calculateSim(jellyfishSim,0.9,remaining,0,filtering3,"phaseBt.txt")

In [ ]:
sentences,distinct_words= createVec(title_to_refs.keys(),l_values)
changes3 , remaining3 = calculateSim(tfIdfSim,0.8,remaining2,0,filtering2,"phaseBt.txt")

In [ ]:
changes4 , remaining4 = calculateSim(jellyfishSim,0.86,remaining3,0,filtering,"phaseBt.txt")

Table-data

In [ ]:
l_values , c_values = dlv , dcv

In [ ]:
changes5 , remaining5 = phaseA(remaining4,title_to_codes,l_values)

In [ ]:
sentences,distinct_words= createVec(title_to_refs.keys(),l_values)
changes6 , remaining6 = calculateSim(tfIdfSim,0.8,remaining5,0,filtering2,"phaseBtd.txt")

In [ ]:
model = trainW2V(l_values)
changes7 , remaining7 = calculateSim(myWord2vec,0.8,remaining6,0,filtering2,"phaseBtd.txt")

2ο task: Relevant σχέσεις

In [ ]:
def getCurrRelations(files): #function that returns a dictionary with the current relations for every reference
  # Create an empty dictionary
  currentRel = {}
  for title in title_to_refs.keys():
    currentRel[title] = []
  # Iterate over the files
  for f in files:
      # Open each file
      with open(f, "r") as file:
          # Read the lines
          lines = file.readlines()
          # Process each line
          for line in lines:
              # Cut the line based on " ------> " delimiter
              extracted_line = line.split(" ------> ")[0]
              # Extract the "A : B" part
              A, B = extracted_line.split(" : ")
              currentRel[A].append(B.strip())
  return currentRel

In [ ]:
currentRel = getCurrRelations(["phaseA.txt", "phaseB.txt"])
title_to_refs , url_to_refs = dbTTR , dbUTR
title_to_codes = createTTC(title_to_refs,url_to_refs)
l_values , c_values = dlv , dcv

Συνάρτηση που εντοπίζει σχέσεις, είτε βάσει ομοιότητας είτε αν υπάρχει ο κωδικός μέσα στον τίτλο.

In [ ]:
from tqdm import tqdm
def relevanceCon(func,filtering,threshold,file):
  titles=[]
  with open(file, "a") as f:
    for title in tqdm(title_to_refs.keys()):
      t= filtering(title)
      for j,label in enumerate(l_values):
        l=filtering(label)
        if label not in currentRel[title]:
          sim = func(t,l)
          # either the similarity is high or the code of label exist in the title of the reference
          if sim >= threshold or (c_values[j] in title.split()):
            f.write("{} : {}\n".format(title,label))
            currentRel[title].append(label)
            titles.append(title)
  return titles

In [ ]:
added1 = relevanceCon(jellyfishSim,filtering3,0.85,"relevantDatabase.txt")

In [ ]:
sentences,distinct_words= createVec(title_to_refs.keys(),l_values)
added2 = relevanceCon(tfIdfSim,filtering2,0.7,"relevantDatabase.txt")

In [ ]:
model = trainW2V(l_values)
added3 = relevanceCon(myWord2vec,filtering2,0.75,"relevantDatabase.txt")

In [ ]:
title_to_refs , url_to_refs = tableTTR , tableUTR
title_to_codes = createTTC(title_to_refs,url_to_refs)
l_values , c_values = tlv , tcv
currentRel = getCurrRelations(["phaseAt.txt", "phaseBt.txt"])

In [ ]:
added1 = relevanceCon(jellyfishSim,filtering3,0.85,"relevantTable.txt")
sentences,distinct_words= createVec(title_to_refs.keys(),l_values)
added2 = relevanceCon(tfIdfSim,filtering2,0.7,"relevantTable.txt")
model = trainW2V(l_values)
added3 = relevanceCon(myWord2vec,filtering2,0.75,"relevantTable.txt")

 42%|████▏     | 44/106 [00:03<00:05, 11.39it/s]<ipython-input-34-0cb9352433d0>:13: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
100%|██████████| 106/106 [00:07<00:00, 14.37it/s]


Αποθήκευση όλων των τριπλέτων με την ιδιότητα hasURI σε ένα αρχείο, και τις τριπλέτες με την ιδιότητα seeAlso σε άλλο αρχείο

In [ ]:
def getCodeByLabel(l,l_values,c_values):
  for i,label in enumerate(l_values):
    if label == l:
      return c_values[i]
def createTheFinalFiles(files,prop,title_to_refs,l_values,c_values,filename):
  with open(filename,'a') as output:
      for f in files:
      # Open each file
        with open(f, "r") as file:
            # Read the lines
            lines = file.readlines()
            # Process each line
            for line in lines:
                # Cut the line based on " ------> " delimiter
                extracted_line = line.split(" ------> ")[0]
                # Extract the "A : B" part
                t, l = extracted_line.split(" : ")
                code = "estat:"+ getCodeByLabel(l.strip(),l_values,c_values)
                for ref in title_to_refs[t]:
                  output.write("estatData:" + ref +" " + prop + " " + code + " ." + "\n")

In [ ]:
title_to_refs1 = dbTTR
l_values1 , c_values1 = dlv , dcv
title_to_refs2 = tableTTR
l_values2 , c_values2 = tlv , tcv

In [ ]:
#hasURI
createTheFinalFiles(["phaseA.txt","phaseB.txt"],"estat:hasURI",title_to_refs1,l_values1,c_values1,"hasURItriples.ttl") #database -> data
createTheFinalFiles(["phaseAt.txt","phaseBt.txt"],"estat:hasURI",title_to_refs2,l_values2,c_values2,"hasURItriples.ttl") #table -> tb
createTheFinalFiles(["phaseAtd.txt","phaseBtd.txt"],"estat:hasURI",title_to_refs2,l_values1,c_values1,"hasURItriples.ttl") #table -> data
#seeAlso
createTheFinalFiles(["relevantDatabase.txt"],"estat:seeAlso",title_to_refs1,l_values1,c_values1,"seeAlsotriples.ttl") #database
createTheFinalFiles(["relevantTable.txt"],"estat:seeAlso",title_to_refs2,l_values2,c_values2,"seeAlsotriples.ttl") #table

Δημιουργώ αρχείο με τις συνδέσεις του hasURI σε clickable format για να μπορούν να ελεγθούν

In [ ]:
with open("hasURIcheckFile.txt",'w') as out:
  with open("hasURItriples.ttl",'r') as input:
    lines = input.readlines()
    for line in lines:
      # Remove prefixes
      modified_format = line.replace("estatData:", "http://lod.csd.auth.gr:7200/resource?uri=https://ec.europa.eu/eurostat/NLP4StatRef/knowledge/")
      # Replace "estat:hasURI" with a :
      modified_format = modified_format.replace("estat:hasURI", " : ")
      # Replace the second occurrence of "estat:" with a colon (:)
      modified_format = modified_format.replace("estat:", "http://lod.csd.auth.gr:7200/resource?uri=https://ec.europa.eu/eurostat/NLP4StatRef/ontology/")
      out.write(modified_format)
  input.close()
out.close()